In [2]:
import re
import csv
import nltk
import statistics

nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger_eng')

from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords, wordnet
from nltk.translate.bleu_score import sentence_bleu

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


In [3]:
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()
stop_words = set(stopwords.words('english'))


def text_cleaning(text: str) -> str:
    text = str(text)

    text = text.encode("ascii", "ignore").decode()

    text = re.sub(r'[^\w\s]', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip().lower()

    # Tokenization
    words = nltk.word_tokenize(text)

    # POS tagging and Lemmatization
    pos_tags = nltk.pos_tag(words)
    pos_map = {'N': wordnet.NOUN, 'V': wordnet.VERB, 'R': wordnet.ADV, 'J': wordnet.ADJ}
    lemmatized_words = [lemmatizer.lemmatize(word, pos_map.get(pos[0], wordnet.NOUN)) for word, pos in pos_tags]

    # Stemming
    stemmed_words = [stemmer.stem(word) for word in lemmatized_words]

    return ' '.join(stemmed_words)

In [4]:
column_manual = []
column_llama = []
column_qwen = []
column_qwen_coder = []


with open('input-v1.csv', 'r', encoding='utf-8') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    next(csv_reader)
    idx = 0
    for row in csv_reader:
        column_manual.append(text_cleaning(row[1]))
        column_llama.append(text_cleaning(row[2]))
        column_qwen.append(text_cleaning(row[3]))
        column_qwen_coder.append(text_cleaning(row[4]))

In [5]:
def compute_word_stats(columns):
    results = {}
    for model_name, column in columns.items():
        word_counts = [len(sentence.split()) for sentence in column]
        average = sum(word_counts) / len(column)
        median = statistics.median(word_counts)
        results[model_name] = {"average": average, "median": median}
        print(f"{model_name} - Average Words: {average}, Median Words: {median}")
    return results

results_word_stats = compute_word_stats({"Manual": column_manual, "Llama": column_llama, "Qwen": column_qwen, "Qwen-Coder": column_qwen_coder})

Manual - Average Words: 8.58252427184466, Median Words: 5.5
Llama - Average Words: 27.87864077669903, Median Words: 11.5
Qwen - Average Words: 14.266990291262136, Median Words: 2.5
Qwen-Coder - Average Words: 9.37864077669903, Median Words: 0.0


In [6]:
from nltk.translate.bleu_score import sentence_bleu
import statistics

def calculate_bleu_stats(column_manual, column_model, weights):
    scores = [sentence_bleu([column_manual[i]], column_model[i], weights=weights) for i in range(len(column_manual))]

    average = round(sum(scores) / len(scores), 4)
    median = round(statistics.median(scores), 4)
    highest = round(max(scores), 4)
    lowest = round(min(scores), 4)
    overall_range = round(highest - lowest, 4)

    # Categorization
    below_03 = [s for s in scores if s < 0.3]
    between_03_05 = [s for s in scores if 0.3 <= s < 0.5]
    above_05 = [s for s in scores if s >= 0.5]

    total = len(scores)

    def get_range(category_scores):
        return round(max(category_scores) - min(category_scores), 4) if category_scores else 0.0

    range_below_03 = get_range(below_03)
    range_between_03_05 = get_range(between_03_05)
    range_above_05 = get_range(above_05)

    return {
        "average": average,
        "median": median,
        "highest": highest,
        "lowest": lowest,
        "overall_range": overall_range,
        "scores": scores,
        "distribution": {
            "<0.3": {
                "count": len(below_03),
                "percentage": round((len(below_03) / total) * 100, 2),
                "range": range_below_03
            },
            "0.3 to <0.5": {
                "count": len(between_03_05),
                "percentage": round((len(between_03_05) / total) * 100, 2),
                "range": range_between_03_05
            },
            ">=0.5": {
                "count": len(above_05),
                "percentage": round((len(above_05) / total) * 100, 2),
                "range": range_above_05
            }
        }
    }

def compute_bleu_scores(column_manual, column_llama, column_qwen, column_qwen_coder):
    models = {
        "Llama": column_llama,
        "Qwen": column_qwen,
        "Qwen-Coder": column_qwen_coder
    }
    weights_list = {
        "BLEU-1": (1.0, 0, 0, 0),
        "BLEU-2": (0.5, 0.5),
        "BLEU-3": (0.33, 0.33, 0.33),
        "BLEU-4": (0.25, 0.25, 0.25, 0.25)
    }

    results = {}
    for model_name, column_model in models.items():
        results[model_name] = {}
        for bleu_name, weights in weights_list.items():
            stats = calculate_bleu_stats(column_manual, column_model, weights)
            results[model_name][bleu_name] = stats

            print(f"{model_name} {bleu_name} - Avg: {stats['average']}, Median: {stats['median']}, "
                  f"Highest: {stats['highest']}, Lowest: {stats['lowest']}, Overall Range: {stats['overall_range']}")
            print(f"  Distribution: ")
            for category, data in stats["distribution"].items():
                print(f"    {category}: Count = {data['count']}, Percentage = {data['percentage']}%, Range = {data['range']}")

    return results

# Compute BLEU scores
results = compute_bleu_scores(column_manual, column_llama, column_qwen, column_qwen_coder)

Llama BLEU-1 - Avg: 0.3887, Median: 0.2059, Highest: 1.0, Lowest: 0, Overall Range: 1.0
  Distribution: 
    <0.3: Count = 113, Percentage = 54.85%, Range = 0.2989
    0.3 to <0.5: Count = 14, Percentage = 6.8%, Range = 0.1617
    >=0.5: Count = 79, Percentage = 38.35%, Range = 0.5
Llama BLEU-2 - Avg: 0.3736, Median: 0.1759, Highest: 1.0, Lowest: 0, Overall Range: 1.0
  Distribution: 
    <0.3: Count = 115, Percentage = 55.83%, Range = 0.2673
    0.3 to <0.5: Count = 16, Percentage = 7.77%, Range = 0.1603
    >=0.5: Count = 75, Percentage = 36.41%, Range = 0.4951


/usr/local/lib/python3.11/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.11/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.11/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_

Llama BLEU-3 - Avg: 0.3671, Median: 0.1674, Highest: 1.0, Lowest: 0, Overall Range: 1.0
  Distribution: 
    <0.3: Count = 116, Percentage = 56.31%, Range = 0.2789
    0.3 to <0.5: Count = 19, Percentage = 9.22%, Range = 0.1945
    >=0.5: Count = 71, Percentage = 34.47%, Range = 0.4967
Llama BLEU-4 - Avg: 0.3554, Median: 0.1481, Highest: 1.0, Lowest: 0, Overall Range: 1.0
  Distribution: 
    <0.3: Count = 119, Percentage = 57.77%, Range = 0.298
    0.3 to <0.5: Count = 19, Percentage = 9.22%, Range = 0.1904
    >=0.5: Count = 68, Percentage = 33.01%, Range = 0.4846
Qwen BLEU-1 - Avg: 0.3599, Median: 0.0, Highest: 1.0, Lowest: 0, Overall Range: 1.0
  Distribution: 
    <0.3: Count = 120, Percentage = 58.25%, Range = 0.2591
    0.3 to <0.5: Count = 11, Percentage = 5.34%, Range = 0.1682
    >=0.5: Count = 75, Percentage = 36.41%, Range = 0.4928
Qwen BLEU-2 - Avg: 0.3532, Median: 0.0, Highest: 1.0, Lowest: 0, Overall Range: 1.0
  Distribution: 
    <0.3: Count = 121, Percentage = 58.74%,

In [7]:
def calculate_bleu_stats(bleu, llama_bleu_scores, qwen_bleu_scores, qwen_coder_bleu_scores, column_manual, column_llama, column_qwen, column_qwen_coder):
    def calculate_filtered_stats(condition):
        filtered_word_counts = {"manual": [], "llama": [], "qwen": [], "qwen_coder": []}

        for i in range(len(llama_bleu_scores)):
            if condition(llama_bleu_scores[i], qwen_bleu_scores[i], qwen_coder_bleu_scores[i]):
                filtered_word_counts["manual"].append(len(column_manual[i].split()))
                filtered_word_counts["llama"].append(len(column_llama[i].split()))
                filtered_word_counts["qwen"].append(len(column_qwen[i].split()))
                filtered_word_counts["qwen_coder"].append(len(column_qwen_coder[i].split()))

        count = len(filtered_word_counts["manual"])
        if count == 0:
            print("No matching entries found.")
            return

        print(count)
        for key in filtered_word_counts:
            print("Average ", key, ":", sum(filtered_word_counts[key]) / count)
            print("Median ", key, ":", statistics.median(filtered_word_counts[key]))

    print(f"## Low BLEU-{bleu} Scores (<0.3)")
    calculate_filtered_stats(lambda l, q, qc: l < 0.3 and q < 0.3 and qc < 0.3)

    print("-----------------------------------------------------")

    print(f"## High BLEU-{bleu} Scores (>0.5)")
    calculate_filtered_stats(lambda l, q, qc: l > 0.5 and q > 0.5 and qc > 0.5)


calculate_bleu_stats(2, results['Llama']['BLEU-2']['scores'], results['Qwen']['BLEU-2']['scores'], results['Qwen-Coder']['BLEU-2']['scores'], column_manual, column_llama, column_qwen, column_qwen_coder)

print("\n#######################################################\n")

calculate_bleu_stats(4, results['Llama']['BLEU-4']['scores'], results['Qwen']['BLEU-4']['scores'], results['Qwen-Coder']['BLEU-4']['scores'], column_manual, column_llama, column_qwen, column_qwen_coder)

## Low BLEU-2 Scores (<0.3)
98
Average  manual : 3.8877551020408165
Median  manual : 0.0
Average  llama : 36.03061224489796
Median  llama : 4.5
Average  qwen : 15.60204081632653
Median  qwen : 0.0
Average  qwen_coder : 10.60204081632653
Median  qwen_coder : 0.0
-----------------------------------------------------
## High BLEU-2 Scores (>0.5)
49
Average  manual : 11.877551020408163
Median  manual : 9
Average  llama : 13.306122448979592
Median  llama : 10
Average  qwen : 12.489795918367347
Median  qwen : 9
Average  qwen_coder : 12.244897959183673
Median  qwen_coder : 10

#######################################################

## Low BLEU-4 Scores (<0.3)
102
Average  manual : 4.147058823529412
Median  manual : 0.0
Average  llama : 35.833333333333336
Median  llama : 5.0
Average  qwen : 15.872549019607844
Median  qwen : 0.0
Average  qwen_coder : 10.264705882352942
Median  qwen_coder : 0.0
-----------------------------------------------------
## High BLEU-4 Scores (>0.5)
46
Average  manual